 ## Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Loading Data of New York

In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

## Transforming Data into Dataframe

In [3]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods_data = newyork_data['features']

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

### Slicing Data to be specific to Manhattan

In [4]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


### Getting coordinates of Manhattan

In [5]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


### Generating a map of Manhattan with Neighborhood Clusters

In [6]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Exploring Neighborhoods With Foursquare API Data Pull

In [7]:
CLIENT_ID = '4E2PGABQUIRG5I3AOBZ43HLYXYSJ5RC53AIWBP1PEP2UPWYW' # your Foursquare ID
CLIENT_SECRET = 'NLBLXBUMO00PP3CKTX2OIFC0KHZDV1AI3ECIAR1JVAYZUMQJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


### CoffeeShops Dataframe Creation

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=50, categoryId='4bf58dd8d48988d1e0931735'):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit,
        categoryId)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [9]:
manhattan_coffee_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [10]:
print(manhattan_coffee_venues.shape)
manhattan_coffee_venues.head()

(1265, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Dunkin Donuts,40.874645,-73.909966,Coffee Shop
1,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
2,Marble Hill,40.876551,-73.91066,Dunkin',40.879308,-73.905066,Donut Shop
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.873234,-73.908730,Coffee Shop


## Bookstore Dataframe Creation

### For this dataframe we replace the category id to the two for bookstore we are searching on (Bookstores & Used Bookstore).


In [11]:
def getBookVenues(names, latitudes, longitudes, radius=500, limit=50, categoryId='4bf58dd8d48988d114951735,52f2ab2ebcbc57f1066b8b30'):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit,
        categoryId)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [12]:
manhattan_book_venues = getBookVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [13]:
print(manhattan_book_venues.shape)
manhattan_book_venues.head()

(287, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,東方書店,40.716022,-73.997581,Bookstore
1,Chinatown,40.715618,-73.994279,Aeon Bookstore,40.713874,-73.990883,Bookstore
2,Chinatown,40.715618,-73.994279,Oriental Culture Enterprises (Eastern Bookstore),40.715949,-73.997345,Bookstore
3,Chinatown,40.715618,-73.994279,Xinhua Bookstore,40.715854,-73.997250,Bookstore
4,Chinatown,40.715618,-73.994279,Ming Fay Book Store,40.715233,-73.998596,Bookstore


In [14]:
#Let's check to see how many bookstores were returned for each neighborhood:
manhattan_book_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,2,2,2,2,2,2
Carnegie Hill,6,6,6,6,6,6
Central Harlem,2,2,2,2,2,2
Chelsea,6,6,6,6,6,6
Chinatown,11,11,11,11,11,11
Civic Center,13,13,13,13,13,13
Clinton,4,4,4,4,4,4
East Village,13,13,13,13,13,13
Financial District,3,3,3,3,3,3


In [15]:
#Let's check to see how many coffeeshops were returned for each neighborhood:

manhattan_coffee_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,28,28,28,28,28,28
Carnegie Hill,34,34,34,34,34,34
Central Harlem,6,6,6,6,6,6
Chelsea,48,48,48,48,48,48
Chinatown,48,48,48,48,48,48
Civic Center,49,49,49,49,49,49
Clinton,44,44,44,44,44,44
East Harlem,8,8,8,8,8,8
East Village,48,48,48,48,48,48


# Clustering Neighborhoods

### We will cluster Manhattan neighborhoods based off their venues similiarity (all venues).

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


### We calculate the mean of frequency of each venue type for each neighborhood.

In [18]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rin

In [19]:
#grouping by top 10 most common venues
num_top_venues = 10

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.10
1          Hotel  0.07
2    Coffee Shop  0.06
3            Gym  0.06
4  Memorial Site  0.04
5  Shopping Mall  0.04
6     Playground  0.03
7  Boat or Ferry  0.03
8   Gourmet Shop  0.03
9          Plaza  0.03


----Carnegie Hill----
                 venue  freq
0          Coffee Shop  0.08
1          Pizza Place  0.04
2                 Café  0.04
3          Yoga Studio  0.03
4    French Restaurant  0.03
5  Japanese Restaurant  0.03
6            Wine Shop  0.03
7   Italian Restaurant  0.03
8       Cosmetics Shop  0.03
9            Bookstore  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Chinese Restaurant  0.05
2  American Restaurant  0.05
3                  Bar  0.05
4   Seafood Restaurant  0.05
5       Cosmetics Shop  0.05
6    French Restaurant  0.05
7  Fried Chicken Joint  0.05
8         Dessert Shop  0.02
9          Music Venue  0.02


----Chelsea----
                

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Shopping Mall,Memorial Site,Plaza,Playground,Boat or Ferry,Gourmet Shop,Food Court,Mexican Restaurant,Bistro,Pub,Garden
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Yoga Studio,French Restaurant,Wine Shop,Bakery,Bookstore,Gym / Fitness Center,Italian Restaurant,Gym,Japanese Restaurant,Cosmetics Shop,Cocktail Bar,Bar
2,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Cosmetics Shop,Bar,Fried Chicken Joint,Seafood Restaurant,French Restaurant,Café,Grocery Store,Tapas Restaurant,Library,Bookstore,Boutique,Southern / Soul Food Restaurant
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Seafood Restaurant,Hotel,Italian Restaurant,Market,Bakery,French Restaurant,Bookstore,Café,Ice Cream Shop,Cupcake Shop,Cycle Studio,Bar
4,Chinatown,Chinese Restaurant,Dessert Shop,Cocktail Bar,Bakery,Salon / Barbershop,American Restaurant,Optical Shop,Vietnamese Restaurant,Spa,Ice Cream Shop,Mexican Restaurant,Shanghai Restaurant,Coffee Shop,Boutique,Greek Restaurant


In [21]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 1, 1, 2, 2, 1], dtype=int32)

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Sandwich Place,Pizza Place,Department Store,Tennis Stadium,Pharmacy,Diner,Discount Store
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Dessert Shop,Cocktail Bar,Bakery,Salon / Barbershop,American Restaurant,Optical Shop,Vietnamese Restaurant,Spa,Ice Cream Shop,Mexican Restaurant,Shanghai Restaurant,Coffee Shop,Boutique,Greek Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Café,Bakery,Chinese Restaurant,Bank,Mobile Phone Shop,Deli / Bodega,Coffee Shop,Supermarket,Latin American Restaurant,Tapas Restaurant,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Grocery Store,New American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,2,Mexican Restaurant,Café,Lounge,Restaurant,Bakery,Caribbean Restaurant,Pharmacy,Spanish Restaurant,Chinese Restaurant,Frozen Yogurt Shop,American Restaurant,Pizza Place,Wine Bar,Deli / Bodega,Park
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Pizza Place,Café,Deli / Bodega,Coffee Shop,Mexican Restaurant,Sandwich Place,Sushi Restaurant,Bakery,Cocktail Bar,Yoga Studio,School,Chinese Restaurant,Caribbean Restaurant,Indian Restaurant,Liquor Store


In [23]:
manhattan_merged.shape

(40, 20)

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:

import matplotlib.pyplot as plt

Sum_of_squared_distances = []
kclusters = range(1,10)
for k in kclusters:
    km = KMeans(n_clusters=k)
    km = km.fit(manhattan_grouped_clustering)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(kclusters, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

# Analyzing Similar Neighborhood Clusters to determine location for business

### Here we will analyze the clusters of similar neighborhoods to determine their ratios of coffeeshops to bookstores. We use normalization and also create an additional map of neighborhooods with an overlay of bookstores and coffeeshops for our final visualation to determine ideal neighborhoods for our recommendation.

### Cluster 1

In [25]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,Chinatown,Chinese Restaurant,Dessert Shop,Cocktail Bar,Bakery,Salon / Barbershop,American Restaurant,Optical Shop,Vietnamese Restaurant,Spa,Ice Cream Shop,Mexican Restaurant,Shanghai Restaurant,Coffee Shop,Boutique,Greek Restaurant
22,Little Italy,Bakery,Chinese Restaurant,Café,Bubble Tea Shop,Coffee Shop,Italian Restaurant,Ice Cream Shop,Seafood Restaurant,Sandwich Place,Salon / Barbershop,Thai Restaurant,Hotel,Mediterranean Restaurant,Women's Store,Wine Bar


### Cluster 2

In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Marble Hill,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Sandwich Place,Pizza Place,Department Store,Tennis Stadium,Pharmacy,Diner,Discount Store
2,Washington Heights,Café,Bakery,Chinese Restaurant,Bank,Mobile Phone Shop,Deli / Bodega,Coffee Shop,Supermarket,Latin American Restaurant,Tapas Restaurant,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Grocery Store,New American Restaurant
5,Manhattanville,Coffee Shop,Seafood Restaurant,Bar,Mexican Restaurant,Chinese Restaurant,Italian Restaurant,Lounge,Bike Trail,Climbing Gym,Gastropub,Falafel Restaurant,Food & Drink Shop,Sushi Restaurant,Spanish Restaurant,Scenic Lookout
6,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Cosmetics Shop,Bar,Fried Chicken Joint,Seafood Restaurant,French Restaurant,Café,Grocery Store,Tapas Restaurant,Library,Bookstore,Boutique,Southern / Soul Food Restaurant
8,Upper East Side,Italian Restaurant,Bakery,Coffee Shop,Gym / Fitness Center,Yoga Studio,Hotel,Juice Bar,French Restaurant,Exhibit,Spa,Pizza Place,Salad Place,Burger Joint,Sushi Restaurant,Women's Store
9,Yorkville,Italian Restaurant,Gym,Bar,Coffee Shop,Deli / Bodega,Sushi Restaurant,Pizza Place,Wine Shop,Japanese Restaurant,Diner,Mexican Restaurant,Pub,Park,Bagel Shop,Chinese Restaurant
10,Lenox Hill,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Pizza Place,Cocktail Bar,Gym,Gym / Fitness Center,Burger Joint,Gift Shop,Salon / Barbershop,Thai Restaurant,Salad Place,Art Gallery,Mexican Restaurant
12,Upper West Side,Italian Restaurant,Coffee Shop,Bar,Mediterranean Restaurant,Dessert Shop,Wine Bar,Bakery,Indian Restaurant,Ice Cream Shop,Thai Restaurant,Seafood Restaurant,Café,Middle Eastern Restaurant,Pub,Sports Bar
13,Lincoln Square,Plaza,Café,Gym / Fitness Center,Concert Hall,Performing Arts Venue,Theater,Italian Restaurant,American Restaurant,Coffee Shop,Wine Shop,Grocery Store,Mediterranean Restaurant,Cycle Studio,Gym,Park
14,Clinton,Theater,Gym / Fitness Center,Coffee Shop,American Restaurant,Cocktail Bar,Sandwich Place,Gym,Italian Restaurant,Spa,Wine Shop,Hotel,Mediterranean Restaurant,New American Restaurant,Chinese Restaurant,Taco Place


### Cluster 3

In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
3,Inwood,Mexican Restaurant,Café,Lounge,Restaurant,Bakery,Caribbean Restaurant,Pharmacy,Spanish Restaurant,Chinese Restaurant,Frozen Yogurt Shop,American Restaurant,Pizza Place,Wine Bar,Deli / Bodega,Park
4,Hamilton Heights,Pizza Place,Café,Deli / Bodega,Coffee Shop,Mexican Restaurant,Sandwich Place,Sushi Restaurant,Bakery,Cocktail Bar,Yoga Studio,School,Chinese Restaurant,Caribbean Restaurant,Indian Restaurant,Liquor Store
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Park,Sandwich Place,Latin American Restaurant,Spa,Beer Bar,Donut Shop,Café,Restaurant,Steakhouse,Liquor Store,Gas Station
19,East Village,Bar,Ice Cream Shop,Mexican Restaurant,Cocktail Bar,Pizza Place,Korean Restaurant,Vietnamese Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Wine Bar,Juice Bar,Japanese Restaurant,Speakeasy,Filipino Restaurant
25,Manhattan Valley,Bar,Mexican Restaurant,Park,Spa,Thai Restaurant,Coffee Shop,Yoga Studio,Pizza Place,Deli / Bodega,Korean Restaurant,Latin American Restaurant,Grocery Store,Szechuan Restaurant,Furniture / Home Store,Fried Chicken Joint
36,Tudor City,Park,Café,Mexican Restaurant,Coffee Shop,Deli / Bodega,Sushi Restaurant,Gym,Spanish Restaurant,Thai Restaurant,Greek Restaurant,Seafood Restaurant,Garden,Dog Run,Pizza Place,Vietnamese Restaurant


### Cluster 4

In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
11,Roosevelt Island,Park,Dry Cleaner,Scenic Lookout,Gym,Greek Restaurant,Coffee Shop,Outdoors & Recreation,Liquor Store,Sandwich Place,School,Supermarket,Metro Station,Bubble Tea Shop,Farmers Market,Food & Drink Shop


### Cluster 5

In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
37,Stuyvesant Town,Park,Bar,Gym / Fitness Center,Pet Service,Cocktail Bar,Baseball Field,Coffee Shop,Bistro,Harbor / Marina,Boat or Ferry,Skating Rink,Gas Station,Heliport,Farmers Market,Fountain


In [30]:
manhattan_merged.groupby('Cluster Labels').count()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,,,,
0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
2,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


## Analyzing  coffee & book stores per each cluster with normalization

In [31]:
array5 = [4]
cluster5=manhattan_merged.loc[manhattan_merged['Cluster Labels'].isin(array5)]
cluster5

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
37,Manhattan,Stuyvesant Town,40.731,-73.974052,4,Park,Bar,Gym / Fitness Center,Pet Service,Cocktail Bar,Baseball Field,Coffee Shop,Bistro,Harbor / Marina,Boat or Ferry,Skating Rink,Gas Station,Heliport,Farmers Market,Fountain


In [32]:
array4 = [3]
cluster4=manhattan_merged.loc[manhattan_merged['Cluster Labels'].isin(array4)]
cluster4

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
11,Manhattan,Roosevelt Island,40.76216,-73.949168,3,Park,Dry Cleaner,Scenic Lookout,Gym,Greek Restaurant,Coffee Shop,Outdoors & Recreation,Liquor Store,Sandwich Place,School,Supermarket,Metro Station,Bubble Tea Shop,Farmers Market,Food & Drink Shop


In [33]:
array3 = [2]
cluster3=manhattan_merged.loc[manhattan_merged['Cluster Labels'].isin(array3)]
cluster3

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
3,Manhattan,Inwood,40.867684,-73.921210,2,Mexican Restaurant,Café,Lounge,Restaurant,Bakery,Caribbean Restaurant,Pharmacy,Spanish Restaurant,Chinese Restaurant,Frozen Yogurt Shop,American Restaurant,Pizza Place,Wine Bar,Deli / Bodega,Park
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Pizza Place,Café,Deli / Bodega,Coffee Shop,Mexican Restaurant,Sandwich Place,Sushi Restaurant,Bakery,Cocktail Bar,Yoga Studio,School,Chinese Restaurant,Caribbean Restaurant,Indian Restaurant,Liquor Store
7,Manhattan,East Harlem,40.792249,-73.944182,2,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Park,Sandwich Place,Latin American Restaurant,Spa,Beer Bar,Donut Shop,Café,Restaurant,Steakhouse,Liquor Store,Gas Station
19,Manhattan,East Village,40.727847,-73.982226,2,Bar,Ice Cream Shop,Mexican Restaurant,Cocktail Bar,Pizza Place,Korean Restaurant,Vietnamese Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Wine Bar,Juice Bar,Japanese Restaurant,Speakeasy,Filipino Restaurant
25,Manhattan,Manhattan Valley,40.797307,-73.964286,2,Bar,Mexican Restaurant,Park,Spa,Thai Restaurant,Coffee Shop,Yoga Studio,Pizza Place,Deli / Bodega,Korean Restaurant,Latin American Restaurant,Grocery Store,Szechuan Restaurant,Furniture / Home Store,Fried Chicken Joint
36,Manhattan,Tudor City,40.746917,-73.971219,2,Park,Café,Mexican Restaurant,Coffee Shop,Deli / Bodega,Sushi Restaurant,Gym,Spanish Restaurant,Thai Restaurant,Greek Restaurant,Seafood Restaurant,Garden,Dog Run,Pizza Place,Vietnamese Restaurant


In [34]:
array2 = [1]
cluster2=manhattan_merged.loc[manhattan_merged['Cluster Labels'].isin(array2)]
cluster2.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Sandwich Place,Pizza Place,Department Store,Tennis Stadium,Pharmacy,Diner,Discount Store
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Café,Bakery,Chinese Restaurant,Bank,Mobile Phone Shop,Deli / Bodega,Coffee Shop,Supermarket,Latin American Restaurant,Tapas Restaurant,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Grocery Store,New American Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,1,Coffee Shop,Seafood Restaurant,Bar,Mexican Restaurant,Chinese Restaurant,Italian Restaurant,Lounge,Bike Trail,Climbing Gym,Gastropub,Falafel Restaurant,Food & Drink Shop,Sushi Restaurant,Spanish Restaurant,Scenic Lookout
6,Manhattan,Central Harlem,40.815976,-73.943211,1,African Restaurant,Chinese Restaurant,American Restaurant,Cosmetics Shop,Bar,Fried Chicken Joint,Seafood Restaurant,French Restaurant,Café,Grocery Store,Tapas Restaurant,Library,Bookstore,Boutique,Southern / Soul Food Restaurant
8,Manhattan,Upper East Side,40.775639,-73.960508,1,Italian Restaurant,Bakery,Coffee Shop,Gym / Fitness Center,Yoga Studio,Hotel,Juice Bar,French Restaurant,Exhibit,Spa,Pizza Place,Salad Place,Burger Joint,Sushi Restaurant,Women's Store


In [35]:
array1= [0]
cluster1=manhattan_merged.loc[manhattan_merged['Cluster Labels'].isin(array1)]
cluster1

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Dessert Shop,Cocktail Bar,Bakery,Salon / Barbershop,American Restaurant,Optical Shop,Vietnamese Restaurant,Spa,Ice Cream Shop,Mexican Restaurant,Shanghai Restaurant,Coffee Shop,Boutique,Greek Restaurant
22,Manhattan,Little Italy,40.719324,-73.997305,0,Bakery,Chinese Restaurant,Café,Bubble Tea Shop,Coffee Shop,Italian Restaurant,Ice Cream Shop,Seafood Restaurant,Sandwich Place,Salon / Barbershop,Thai Restaurant,Hotel,Mediterranean Restaurant,Women's Store,Wine Bar


### Calculating Sums and Normalizations per Cluster

In [36]:
df_cluster1=cluster1[['Neighborhood','Cluster Labels']]
df_cluster1

,Neighborhood,Cluster Labels
1,Chinatown,0
22,Little Italy,0


In [37]:
bookcount=manhattan_book_venues.groupby('Neighborhood').count()
bookcount=bookcount.drop(columns=['Neighborhood Latitude', 'Neighborhood Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
bookcount.rename({'Venue': 'Count of Bookstores',}, axis=1, inplace=True)
bookcount.head()

,Count of Bookstores
Neighborhood,
Battery Park City,2
Carnegie Hill,6
Central Harlem,2
Chelsea,6
Chinatown,11


In [38]:
coffeecount=manhattan_coffee_venues.groupby('Neighborhood').count()
coffeecount=coffeecount.drop(columns=['Neighborhood Latitude', 'Neighborhood Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
coffeecount.rename({'Venue': 'Count of Coffee Shops',}, axis=1, inplace=True)
coffeecount.head()

,Count of Coffee Shops
Neighborhood,
Battery Park City,28
Carnegie Hill,34
Central Harlem,6
Chelsea,48
Chinatown,48


In [39]:
combinedcount=pd.concat([bookcount, coffeecount], axis=1, join='inner')
combinedcount

,Count of Bookstores,Count of Coffee Shops
Neighborhood,,
Battery Park City,2,28
Carnegie Hill,6,34
Central Harlem,2,6
Chelsea,6,48
Chinatown,11,48
Civic Center,13,49
Clinton,4,44
East Village,13,48
Financial District,3,50


In [40]:
manhattan_merged.head(2)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Sandwich Place,Pizza Place,Department Store,Tennis Stadium,Pharmacy,Diner,Discount Store
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Dessert Shop,Cocktail Bar,Bakery,Salon / Barbershop,American Restaurant,Optical Shop,Vietnamese Restaurant,Spa,Ice Cream Shop,Mexican Restaurant,Shanghai Restaurant,Coffee Shop,Boutique,Greek Restaurant


In [41]:
master=manhattan_merged[['Neighborhood','Cluster Labels']]
master.set_index('Neighborhood', inplace=True)
master

,Cluster Labels
Neighborhood,
Marble Hill,1
Chinatown,0
Washington Heights,1
Inwood,2
Hamilton Heights,2
Manhattanville,1
Central Harlem,1
East Harlem,2
Upper East Side,1


In [42]:
df=pd.concat([master,combinedcount], axis=1)
df=df.fillna(0)
df

,Cluster Labels,Count of Bookstores,Count of Coffee Shops
Marble Hill,1,0.0,0.0
Chinatown,0,11.0,48.0
Washington Heights,1,0.0,0.0
Inwood,2,2.0,7.0
Hamilton Heights,2,0.0,0.0
Manhattanville,1,0.0,0.0
Central Harlem,1,2.0,6.0
East Harlem,2,0.0,0.0
Upper East Side,1,10.0,34.0
Yorkville,1,2.0,19.0


In [43]:
df=df.groupby("Cluster Labels").sum()

In [44]:
df.insert(2,"Count of Neighborhoods",[2,30,6,1,1])

In [45]:
df["Normalized Bookstores"]=df['Count of Bookstores']/231

In [46]:
df["Normalized Coffee Shops"]=df['Count of Coffee Shops']/962

In [47]:
df['Rating']=df['Normalized Bookstores']*df['Normalized Coffee Shops']

In [48]:
df['Percentage B to C']=df['Count of Bookstores']/df['Count of Coffee Shops']*100

In [49]:
df

,Count of Bookstores,Count of Coffee Shops,Count of Neighborhoods,Normalized Bookstores,Normalized Coffee Shops,Rating,Percentage B to C
Cluster Labels,,,,,,,
0,30.0,96.0,2,0.129870,0.099792,0.012960,31.250000
1,231.0,962.0,30,1.000000,1.000000,1.000000,24.012474
2,25.0,113.0,6,0.108225,0.117464,0.012713,22.123894
3,1.0,3.0,1,0.004329,0.003119,0.000014,33.333333
4,0.0,0.0,1,0.000000,0.000000,0.000000,NaN


### Mapping Manhattan Clusters with Bookstores and Coffee Shops

In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [69]:


# instantiate a feature group for the bookstores in the dataframe
bookstoresmap = folium.map.FeatureGroup()

# loop through the bookstores and add each to the  feature group
for lat, lng, in zip(manhattan_book_venues['Venue Latitude'], manhattan_book_venues['Venue Longitude']):
    bookstoresmap.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=4, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
map_clusters.add_child(bookstoresmap)

# instantiate a feature group for the bookstores in the dataframe
coffeeshopmap = folium.map.FeatureGroup()

# loop through the coffe shops and add each to the  feature group
for lat, lng, in zip(manhattan_coffee_venues['Venue Latitude'], manhattan_coffee_venues['Venue Longitude']):
    coffeeshopmap.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=4, # define how big you want the circle markers to be
            color='black',
            fill=True,
            fill_color='white',
            fill_opacity=0.3
        )
    )

# add incidents to map
map_clusters.add_child(coffeeshopmap)